In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Налаштування для коректного відображення українських символів
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
import sqlite3
import matplotlib.pyplot as plt

# Підключення до бази даних
conn = sqlite3.connect('../db.sqlite3')

# SQL запит для об'єднання всіх трьох таблиць
query = """
SELECT 
    oi.id as orderitem_id,
    oi.quantity,
    oi.order_id,
    oi.product_id,
    p.name as product_name,
    p.price as product_price,
    o.datetime as order_datetime
FROM restaurant_orderitem oi
JOIN restaurant_product p ON oi.product_id = p.id
JOIN restaurant_order o ON oi.order_id = o.id
"""

# Виконання запиту та створення DataFrame
df = pd.read_sql_query(query, conn)

# Збереження в CSV файл
df.to_csv('restaurant_data.csv', index=False)

# Перевірка кількості рядків
print(f"Кількість рядків у DataFrame: {len(df)}")
print(f"Розміри DataFrame: {df.shape}")
print("\nПерші 5 рядків:")
print(df.head())

# Закриття з'єднання
conn.close()

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
# Читаємо CSV файл (якщо ще не завантажений)
if 'df' not in locals():
    df = pd.read_csv('restaurant_data.csv')

# Групуємо за назвою продукту та сумуємо кількість
product_quantities = df.groupby('product_name')['quantity'].sum().sort_values(ascending=False)

# Топ 10 найпопулярніших продуктів
top_10_products = product_quantities.head(10)

print("Топ 10 найпопулярніших продуктів за кількістю:")
print(top_10_products)

# Створюємо діаграму пирог
plt.figure(figsize=(12, 8))
colors = plt.cm.Set3(range(len(top_10_products)))

# Створюємо діаграму з відсотками
wedges, texts, autotexts = plt.pie(top_10_products.values, 
                                  labels=top_10_products.index, 
                                  autopct='%1.1f%%',
                                  colors=colors,
                                  startangle=90)

plt.title('Топ 10 найпопулярніших продуктів за кількістю', fontsize=16, fontweight='bold')

# Покращуємо читабельність
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

plt.axis('equal')
plt.tight_layout()
plt.show()

# Виводимо статистику
total_quantity = product_quantities.sum()
print(f"\nЗагальна кількість проданих продуктів: {total_quantity}")
print(f"Частка топ-10 продуктів: {top_10_products.sum() / total_quantity * 100:.1f}%")

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
# Обчислюємо ціну за одиницю товару (Item Price)
df['item_price'] = df['product_price'] * df['quantity']

# Групуємо за назвою продукту та сумуємо дохід
product_revenue = df.groupby('product_name')['item_price'].sum().sort_values(ascending=False)

# Топ 10 продуктів за доходом
top_10_revenue = product_revenue.head(10)

print("Топ 10 продуктів за доходом:")
for i, (product, revenue) in enumerate(top_10_revenue.items(), 1):
    print(f"{i}. {product}: ${revenue:.2f}")

# Створюємо діаграму пирог для доходу
plt.figure(figsize=(12, 8))
colors = plt.cm.Pastel1(range(len(top_10_revenue)))

# Створюємо діаграму з відсотками
wedges, texts, autotexts = plt.pie(top_10_revenue.values, 
                                  labels=top_10_revenue.index, 
                                  autopct='%1.1f%%',
                                  colors=colors,
                                  startangle=90)

plt.title('Топ 10 продуктів за доходом', fontsize=16, fontweight='bold')

# Покращуємо читабельність
for autotext in autotexts:
    autotext.set_color('black')
    autotext.set_fontweight('bold')
    autotext.set_fontsize(10)

plt.axis('equal')
plt.tight_layout()
plt.show()

# Виводимо статистику
total_revenue = product_revenue.sum()
print(f"\nЗагальний дохід ресторану: ${total_revenue:.2f}")
print(f"Дохід від топ-10 продуктів: ${top_10_revenue.sum():.2f}")
print(f"Частка топ-10 продуктів у доході: {top_10_revenue.sum() / total_revenue * 100:.1f}%")

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
# Конвертуємо datetime в pandas datetime
df['order_datetime'] = pd.to_datetime(df['order_datetime'])

# Витягуємо годину з datetime
df['order_hour'] = df['order_datetime'].dt.hour

# Групуємо за годиною та сумуємо дохід
hourly_revenue = df.groupby('order_hour')['item_price'].sum()

# Створюємо стовпчасту діаграму
plt.figure(figsize=(14, 8))
bars = plt.bar(hourly_revenue.index, hourly_revenue.values, 
               color='skyblue', edgecolor='navy', alpha=0.7)

plt.title('Дохід ресторану за годинами дня', fontsize=16, fontweight='bold')
plt.xlabel('Година дня', fontsize=12)
plt.ylabel('Дохід ($)', fontsize=12)
plt.xticks(range(24))
plt.grid(axis='y', alpha=0.3)

# Додаємо значення на стовпчики
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 50,
             f'${height:.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Виводимо статистику
print("Дохід за кожну годину дня:")
for hour, revenue in hourly_revenue.items():
    print(f"Година {hour:2d}: ${revenue:,.2f}")

print(f"\nНайприбутковіша година: {hourly_revenue.idxmax()} (${hourly_revenue.max():.2f})")
print(f"Найменш прибуткова година: {hourly_revenue.idxmin()} (${hourly_revenue.min():.2f})")
print(f"Середній дохід за годину: ${hourly_revenue.mean():.2f}")

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
# Витягуємо день тижня з datetime
df['order_day_of_week'] = df['order_datetime'].dt.day_name()

# Визначаємо порядок днів тижня
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_names_ua = ['Понеділок', 'Вівторок', 'Середа', 'Четвер', 'П\'ятниця', 'Субота', 'Неділя']

# Групуємо за днем тижня та сумуємо дохід
daily_revenue = df.groupby('order_day_of_week')['item_price'].sum()

# Сортуємо за порядком днів тижня
daily_revenue = daily_revenue.reindex(day_order)

# Створюємо стовпчасту діаграму
plt.figure(figsize=(12, 8))
colors = ['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#FF99CC', '#99CCFF', '#FFB366']
bars = plt.bar(day_names_ua, daily_revenue.values, 
               color=colors, edgecolor='black', alpha=0.8)

plt.title('Дохід ресторану за днями тижня', fontsize=16, fontweight='bold')
plt.xlabel('День тижня', fontsize=12)
plt.ylabel('Дохід ($)', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

# Додаємо значення на стовпчики
for i, bar in enumerate(bars):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 100,
             f'${height:,.0f}', ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# Виводимо статистику
print("Дохід за кожен день тижня:")
for day_en, day_ua in zip(day_order, day_names_ua):
    if day_en in daily_revenue.index:
        revenue = daily_revenue[day_en]
        print(f"{day_ua}: ${revenue:,.2f}")

# Знаходимо найкращий і найгірший дні
best_day_en = daily_revenue.idxmax()
worst_day_en = daily_revenue.idxmin()
best_day_ua = day_names_ua[day_order.index(best_day_en)]
worst_day_ua = day_names_ua[day_order.index(worst_day_en)]

print(f"\nНайприбутковіший день: {best_day_ua} (${daily_revenue.max():,.2f})")
print(f"Найменш прибутковий день: {worst_day_ua} (${daily_revenue.min():,.2f})")
print(f"Середній дохід за день: ${daily_revenue.mean():,.2f}")

# Аналіз weekday vs weekend
weekday_revenue = daily_revenue[day_order[:5]].sum()  # Пн-Пт
weekend_revenue = daily_revenue[day_order[5:]].sum()  # Сб-Нд

print(f"\nДохід у будні дні (Пн-Пт): ${weekday_revenue:,.2f}")
print(f"Дохід у вихідні (Сб-Нд): ${weekend_revenue:,.2f}")
print(f"Співвідношення будні/вихідні: {weekday_revenue/weekend_revenue:.2f}")